# Assignment 2

**Credits**: Federico Ruggeri, Eleonora Mancini, Paolo Torroni

**Keywords**: Sexism Detection, Multi-class Classification, LLMs, Prompting


# Contact

For any doubt, question, issue or help, you can always contact us at the following email addresses:

Teaching Assistants:

* Federico Ruggeri -> federico.ruggeri6@unibo.it
* Eleonora Mancini -> e.mancini@unibo.it

Professor:

* Paolo Torroni -> p.torroni@unibo.it

# Relevant Material

- Tutorial 3
- Huggingface documentation
- Huggingface hub

# Introduction

You are tasked to address the [EDOS Task B](https://github.com/rewire-online/edos) on sexism detection.

## Problem definition

Given an input text sentence, the task is to label the sentence as not-sexist or one of these four sexist categories: (1) threats, (2)  derogation, (3) animosity, (4) prejudiced discussion.

### Examples:

**Text**: *``Schedule a date with her, then don't show up. Then text her "GOTCHA B___H".''*

**Label**: Threats

**Text**: *``That’s completely ridiculous a woman flashing her boobs is not sexual assault in the slightest.''*

**Label**: Not sexist



## Approach

We will tackle the five-class classification task with LLMs.

In particular, we'll consider zero-/few-shot prompting approaches to assess the capability of some popular open-source LLMs on this task.

## Preliminaries

We are going to download LLMs from [Huggingface](https://huggingface.co/).

Many of these open-source LLMs require you to accept their "Community License Agreement" to download them.

In summary:

- If not already, create an account of Huggingface (~2 mins)
- Check a LLM model card page (e.g., [Mistral v3](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3)) and accept its "Community License Agreement".
- Go to your account -> Settings -> Access Tokens -> Create new token -> "Repositories permissions" -> add the LLM model card you want to use.
- Save the token (we'll need it later)

## Setup and dependencies installation

In the following, we will assume that you have 
- created a local python virtual environment - either with python [venv](https://docs.python.org/3/library/venv.html) module or via [uv](https://github.com/astral-sh/uv) (preferred) - with the `ipykernel` or `jupyter` packages pre-installed to start the jupyter kernel;
- a stable internet connection
Nothing else is needed.

We will now download the `pyproject.toml` file specifying the project dependencies.

In [ ]:
import os
import urllib.request
from pathlib import Path


def get_project_root() -> Path:
    """Return the root directory of the project."""
    start_dir = Path.cwd()

    markers = ["assignment2.ipynb"]

    for path in [start_dir, *list(start_dir.parents)]:
        for marker in markers:
            if (path / marker).exists():
                return path

    return start_dir


project_root: Path = get_project_root()

In [ ]:
project_repo: str = "mpreda01/nlp-assignments"
project_branch: str = "main"
pyproject_url = (
    f"https://raw.githubusercontent.com/{project_repo}/{project_branch}/pyproject.toml"
)
lockfile_url = (
    f"https://raw.githubusercontent.com/{project_repo}/{project_branch}/uv.lock"
)
urllib.request.urlretrieve(pyproject_url, project_root / "pyproject.toml")  # noqa: S310
urllib.request.urlretrieve(lockfile_url, project_root / "uv.lock");  # noqa: S310

If using [uv](https://github.com/astral-sh/uv) (recommended) you can now install the dependencies to a local virtual environment at `.venv` simply via
```sh
uv sync --extra assignment2
```

If not, the same can be achieved with the usual python [venv](https://docs.python.org/3/library/venv.html):
```sh
python3 -m venv .venv
source .venv/bin/activate
(.venv) pip install ".[assignment2]" 
```

Make sure to do the above and *restart the kernel* if necessary before proceeding.

In [ ]:
import copy

import pandas as pd
import torch
from huggingface_hub import login, whoami
from sklearn.metrics import f1_score
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

### Huggingface Login

Once we have created an account and an access token, we need to login to Huggingface via code. Add an environment variable `HF_TOKEN` with your token
```sh
export HF_TOKEN="my-huggingface-token-with-the-correct-access-rights"
```
as [expected by huggingface](https://huggingface.co/docs/huggingface_hub/package_reference/environment_variables#hftoken).


In [ ]:
hf_token: str = os.environ.get("HF_TOKEN")

if hf_token:
    try:
        login(token=hf_token, add_to_git_credential=True)
        user = whoami(token=hf_token)
        username = user["name"]
        print(f"Logged in as @{username}")
        print(f"Profile page: https://huggingface.co/{username}")
    except ValueError:
        print("Invalid token.")
    except ImportError:
        print("ipywidgets not installed.")
else:
    print("Warning: 'HF_TOKEN' environment variable not set.")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Logged in as @niccolozanotti
Profile page: https://huggingface.co/niccolozanotti


After login, you can download all models associated with your access token in addition to those that are not protected by an access token.

### Data Loading

Since we are only interested in prompting, we do not require a train dataset.

We have preparared a small test set version of EDOS in our dedicated [Github repository](https://github.com/lt-nlp-lab-unibo/nlp-course-material).

Check the ``Assignment 2/data`` folder.
It contains:

- ``a2_test.csv`` → a small test set of 300 samples.
- ``demonstrations.csv`` -> a batch of 1000 samples for few-shot prompting.

Both datasets contain a balanced number of sexist and not sexist samples.


### Instructions

We require you to:

* **Download** the ``A2/data`` folder.
* **Encode** ``a2_test.csv`` into a ``pandas.DataFrame`` object.

In [ ]:
repo: str = "nlp-unibo/nlp-course-material"
branch: str = "main"
folder_path: str = "2025-2026/Assignment%202/data"
data_dir: Path = project_root / "data"
files = ["a2_test.csv", "demonstrations.csv"]

base_url = f"https://raw.githubusercontent.com/{repo}/{branch}/{folder_path}"
data_dir.mkdir(exist_ok=True)

for filename in files:
    url = f"{base_url}/{filename}"
    destination = data_dir / f"{filename}"
    urllib.request.urlretrieve(url, destination)  # noqa: S310
    print(f"Saved to: {destination}")

Saved to: /Users/niccolozanotti/software/github.com/nlp-project/data/a2_test.csv
Saved to: /Users/niccolozanotti/software/github.com/nlp-project/data/demonstrations.csv
Loaded 300 test samples


In [ ]:
test_df = pd.read_csv("data/a2_test.csv")
print(f"Loaded {len(test_df)} test samples")

# [Task 1 - 0.5 points] Model setup

Once the test data has been loaded, we have to setup the model pipeline for inference.

In particular, we have to:
- Load the model weights from Huggingface
- Quantize the model to fit into a single-GPU limited hardware

## Which LLMs?

The pool of LLMs is ever increasing and it's impossible to keep track of all new entries.

We focus on popular open-source models.

- [Mistral v2](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2)
- [Mistral v3](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3)
- [Llama v3.1](https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct)
- [Phi3-mini](https://huggingface.co/microsoft/Phi-3-mini-4k-instruct)
- [TinyLlama](https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v1.0)
- [DeepSeek-R1](https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Qwen-7B)
- [Qwen3](https://huggingface.co/Qwen/Qwen3-1.7B)

Other open-source models are more than welcome!

### Instructions

In order to get Task 1 points, we require you to:

* Pick 2 model cards from the provided list.
* For each model:
  - Setup a quantization configuration for the model.
  - Load the model via HuggingFace APIs.


### Note

There's a popular library integrated with Huggingface's ``transformers`` to perform quantization.


In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# --- Model 1: DeepSeek ---
MODEL_1_ID = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"

if "model_DeepSeek" not in globals() or model_DeepSeek is None:
    print(f"Loading Model 1: {MODEL_1_ID}")

    tokenizer_DeepSeek = AutoTokenizer.from_pretrained(MODEL_1_ID, force_download=True)
    model_DeepSeek = AutoModelForCausalLM.from_pretrained(
        MODEL_1_ID,
        return_dict=True,
        quantization_config=quantization_config,
        device_map="auto",
    )
    # Attach tokenizer to model object
    model_DeepSeek.tokenizer = tokenizer_DeepSeek

    if tokenizer_DeepSeek.pad_token is None:
        tokenizer_DeepSeek.pad_token = tokenizer_DeepSeek.eos_token

    print("Model 1 loaded successfully!")
else:
    print(f"Model 1 ({MODEL_1_ID}) already loaded, skipping...")


# --- Model 2: Mistral V3---
MODEL_2_ID = "mistralai/Mistral-7B-Instruct-v0.3"

if "model_mistral" not in globals() or model_mistral is None:
    print(f"Loading Model 2: {MODEL_2_ID}")

    tokenizer_mistral = AutoTokenizer.from_pretrained(MODEL_2_ID, force_download=True)
    model_mistral = AutoModelForCausalLM.from_pretrained(
        MODEL_2_ID,
        quantization_config=quantization_config,
        device_map="auto",
        return_dict=True,
    )
    # Attach tokenizer to model object
    model_mistral.tokenizer = tokenizer_mistral

    if tokenizer_mistral.pad_token is None:
        tokenizer_mistral.pad_token = tokenizer_mistral.eos_token

    print("Model 2 loaded successfully!")
else:
    print(f"Model 2 ({MODEL_2_ID}) already loaded, skipping...")

Loading Model 1: deepseek-ai/DeepSeek-R1-Distill-Qwen-7B


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Model 2 loaded successfully!


# [Task 2 - 1.0 points] Prompt setup

Prompting requires an input pre-processing phase where we convert each input example into a specific instruction prompt.


## Prompt Template

Use the following prompt template to process input texts.

In [ ]:
prompt = [
    {"role": "system", "content": "You are an annotator for sexism detection."},
    {
        "role": "user",
        "content": """Your task is to classify input text as not-sexist
         or sexist. If sexist, classify input text according to one
         of the following four categories: threats, derogation,
         animosity, prejudiced discussion.

         Below you find sexist categories definitions:
         Threats: the text expresses intent or desire to harm a woman.
         Derogation: the text describes a woman in a derogative manner.
         Animosity: the text contains slurs or insults towards a woman.
         Prejudiced discussion: the text expresses supports for
         mistreatment of women as individuals.

         Respond only by writing one of the following categories:
         not-sexist, threats, derogation, animosity, prejudiced.

        TEXT: {text}

        ANSWER:
        """,
    },
]

### Instructions

In order to get Task 2 points, we require you to:

* Write a ``prepare_prompts`` function as the one reported below.

In [ ]:
def prepare_prompts(texts, prompt_template, tokenizer):
    """This function format input text samples into instructions prompts.

    Inputs:
      texts: input texts to classify via prompting
      prompt_template: the prompt template provided in this assignment
      tokenizer: the transformers Tokenizer object instance associated
      with the chosen model card

    Outputs:
      input texts to classify in the form of instruction prompts
    """
    prepared_prompts = []

    for text in texts:
        prompt_messages = copy.deepcopy(prompt_template)

        prompt_messages[1]["content"] = prompt_messages[1]["content"].format(text=text)

        # Use the tokenizer's chat template to format the messages
        # This converts the list of message dicts into the model's expected format
        formatted_prompt = tokenizer.apply_chat_template(
            prompt_messages,
            tokenize=False,  # Return string, not token IDs
            add_generation_prompt=True,  # Add the prompt for the model to generate
        )

        prepared_prompts.append(formatted_prompt)

    return prepared_prompts

### Notes

1. You are free to modify the prompt format (**not its content**) as you like depending on your code implementation.

2. Note that the provided prompt has placeholders. You need to format the string to replace placeholders. Huggingface might have dedicated APIs for this.

# [Task 3 - 1.0 points] Inference

We are now ready to define the inference loop where we prompt the model with each pre-processed sample.

### Instructions

In order to get Task 3 points, we require you to:

* Write a ``generate_responses`` function as the one reported below.
* Write a ``process_response`` function as the one reported below.

In [ ]:
# Global label
LABEL_MAP = {
    "not-sexist": 0,
    "threats": 1,
    "derogation": 2,
    "animosity": 3,
    "prejudiced": 4,
}

In [ ]:
def generate_responses(model, prompt_examples):
    """This function implements the inference loop for a LLM model.
    Given a set of examples, the model is tasked to generate
    a response.

    Inputs:
      model: LLM model instance for prompting
      prompt_examples: pre-processed text samples

    Outputs:
      generated responses
    """

    # Access tokenizer from the model object
    tokenizer = model.tokenizer

    responses = []
    model.eval()

    for prompt in tqdm(prompt_examples, desc="Generating responses"):
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            # Define max_new_tokens for generation
            # (e.g., a small number for classification answers)
            max_new_tokens = (
                80 if "deepseek" in model.config.name_or_path.lower() else 40
            )  # deepseek is a reasoning model and needs more tokens

            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                pad_token_id=tokenizer.pad_token_id,
            )

        # Decode only the newly generated tokens
        generated_tokens = outputs[0][inputs["input_ids"].shape[1] :]
        response = tokenizer.decode(generated_tokens, skip_special_tokens=True)

        responses.append(response.strip())

    return responses

In [ ]:
def process_response(response):
    """This function takes a textual response generated by the LLM
    and processes it to map the response to a binary label.

    Inputs:
      response: generated response from LLM

    Outputs:
      parsed classification response.
      Use the following mapping:
      {
        'not-sexist': 0,
        'threats': 1,
        'derogation': 2,
        'animosity': 3,
        'prejudiced': 4
      }
    """
    response_lower = response.lower().strip()

    # Extract text after "ANSWER:" if present
    if "answer:" in response_lower:
        response_lower = response_lower.split("answer:")[-1].strip()

    # For DeepSeek: extract content after </think> if present
    if "</think>" in response_lower:
        response_lower = response_lower.split("</think>")[-1].strip()

    for label, idx in LABEL_MAP.items():
        if label in response_lower:
            return idx
    return 0

In [ ]:
# Get texts as a list
texts = test_df["text"].tolist()

# Prepare prompts
prompts_DeepSeek = prepare_prompts(texts, prompt, tokenizer_DeepSeek)
prompts_mistral = prepare_prompts(texts, prompt, tokenizer_mistral)

# Generate responses
responses_DeepSeek = generate_responses(model_DeepSeek, prompts_DeepSeek)
responses_mistral = generate_responses(model_mistral, prompts_mistral)

Generating responses: 100%|██████████| 300/300 [01:36<00:00,  3.10it/s]


## Notes

1. According to our tests, it should take you ~10 mins to perform full inference on 300 samples on Colab.

# [Task 4 - 0.5 points] Metrics

In order to evaluate selected LLMs, we need to compute performance metrics.

We compute **macro F1-score** and the ratio of failed responses generated by models (**fail-ratio**).

That is, how frequent the LLM fails to follow instructions and provides incorrect responses that do not address the classification task.

In summary, we parse generated responses as follows:
- **0** if 'not-sexist'
- **1** if 'threats'
- **2** if 'derogation'
- **3** if 'animosity'
- **4** if 'prejudiced'
- **0** if the model does not answer in either way

### Instructions

In order to get Task 4 points, we require you to:

* Write a ``compute_metrics`` function as the one reported below.
* Compute metrics for the two selected LLMs.

In [ ]:
def compute_metrics(y_pred, y_true):
    """This function takes predicted and ground-truth labels and compute
    metrics. In particular, this function compute accuracy and
    fail-ratio metrics. This function internally invokes
    `process_response` to compute metrics.

    Inputs:
      y_pred: parsed LLM responses
      y_true: ground-truth binary labels

    Outputs:
      dictionary containing desired metrics
    """
    failures = 0
    parsed_predictions = []

    for response in y_pred:
        response_lower = response.lower().strip()

        # Check if response contains any valid label
        found_label = any(label in response_lower for label in LABEL_MAP)

        if not found_label:
            failures += 1

        parsed_predictions.append(process_response(response))

    fail_ratio = failures / len(y_pred)
    macro_f1 = f1_score(y_true, parsed_predictions, average="macro")

    return {
        "macro_f1": macro_f1,
        "fail_ratio": fail_ratio,
    }

In [ ]:
# Get ground truth labels as integers
y_true = test_df["label_category"].map(LABEL_MAP).tolist()

# Compute metrics
metrics_DeepSeek = compute_metrics(responses_DeepSeek, y_true)
metrics_mistral = compute_metrics(responses_mistral, y_true)

print("DeepSeek:", metrics_DeepSeek)
print("Mistral:", metrics_mistral)

DeepSeek: {'macro_f1': 0.0659217877094972, 'fail_ratio': 0.0}
Mistral: {'macro_f1': 0.3394486079547122, 'fail_ratio': 0.0}


# [Task 5 - 1.0 points] Few-shot Inference

So far, we have tested models in a zero-shot fashion: we provide the input text to classify and instruct the model to generate a response.

We are now interested in performing few-shot prompting to see the impact of providing demonstration examples.

To do so, we slightly change the prompt template as follows.

In [ ]:
prompt = [
    {"role": "system", "content": "You are an annotator for sexism detection."},
    {
        "role": "user",
        "content": """Your task is to classify input text as not-sexist
         or sexist. If sexist, classify input text according to one
         of the following four categories: threats, derogation,
         animosity, prejudiced discussion.

         Below you find sexist categories definitions:
         Threats: the text expresses intent or desire to harm a woman.
         Derogation: the text describes a woman in a derogative manner.
         Animosity: the text contains slurs or insults towards a woman.
         Prejudiced discussion: the text expresses supports for
         mistreatment of women as individuals.

         Respond only by writing one of the following categories:
         not-sexist, threats, derogation, animosity, prejudiced.

        EXAMPLES: {examples}

        TEXT: {text}

        ANSWER:
        """,
    },
]

The new prompt template reports some demonstration examples to instruct the model.

Generally, we provide an equal number of demonstrations per class as shown in the example below.

In [ ]:
prompt = [
    {"role": "system", "content": "You are an annotator for sexism detection."},
    {
        "role": "user",
        "content": """Your task is to classify input text as not-sexist
         or sexist. If sexist, classify input text according to one
         of the following four categories: threats, derogation,
         animosity, prejudiced discussion.

         Below you find sexist categories definitions:
         Threats: the text expresses intent or desire to harm a woman.
         Derogation: the text describes a woman in a derogative manner.
         Animosity: the text contains slurs or insults towards a woman.
         Prejudiced discussion: the text expresses supports for
         mistreatment of women as individuals.

         Respond only by writing one of the following categories:
         not-sexist, threats, derogation, animosity, prejudiced.

         EXAMPLES:
         TEXT: **example 1**
         ANSWER: threats
         TEXT: **example 2**
         ANSWER: not-sexist

         TEXT: {text}

        ANSWER:
        """,
    },
]

## Instructions

In order to get Task 5 points, we require you to:

- Load ``demonstrations.csv`` and encode it into a ``pandas.DataFrame`` object.
- Define a ``build_few_shot_demonstrations`` function as the one reported below.
- Modify ``prepare_prompts`` to support demonstrations.
- Perform few-shot inference as in Task 3.
- Compute metrics as in Task 4.

In [ ]:
def build_few_shot_demonstrations(demonstrations, num_per_class=2):
    """Inputs:
      demonstrations: DataFrame wrapping demonstrations.csv
      num_per_class: number of demonstrations per class

    Outputs:
      list of demonstrations to inject into the prompt template.
    """
    examples = []

    for label in LABEL_MAP:
        # Get samples for this label
        label_samples = demonstrations[demonstrations["label_category"] == label].head(
            num_per_class
        )

        for _, row in label_samples.iterrows():
            examples.append(f"TEXT: {row['text']}\nANSWER: {label}")

    return "\n".join(examples)

In [ ]:
# 1. Load demonstrations
demonstrations_df = pd.read_csv(data_dir / "demonstrations.csv")

# 2. Build demonstrations string
examples_str = build_few_shot_demonstrations(demonstrations_df, num_per_class=4)

# 3. Define few-shot prompt template (with {examples} placeholder filled)
few_shot_prompt = [
    {"role": "system", "content": "You are an annotator for sexism detection."},
    {
        "role": "user",
        "content": f"""Your task is to classify input text as not-sexist
         or sexist. If sexist, classify input text according to one
         of the following four categories: threats, derogation,
         animosity, prejudiced discussion.

         Below you find sexist categories definitions:
         Threats: the text expresses intent or desire to harm a woman.
         Derogation: the text describes a woman in a derogative manner.
         Animosity: the text contains slurs or insults towards a woman.
         Prejudiced discussion: the text expresses supports for
         mistreatment of women as individuals.

         Respond only by writing one of the following categories:
         not-sexist, threats, derogation, animosity, prejudiced.

         EXAMPLES:
{examples_str}

        TEXT: {{text}}

        ANSWER:
        """,
    },
]

# 4. Prepare few-shot prompts
prompts_DeepSeek_fewshot = prepare_prompts(texts, few_shot_prompt, tokenizer_DeepSeek)
prompts_mistral_fewshot = prepare_prompts(texts, few_shot_prompt, tokenizer_mistral)

# 5. Generate responses (few-shot)
responses_DeepSeek_fewshot = generate_responses(
    model_DeepSeek, prompts_DeepSeek_fewshot
)
responses_mistral_fewshot = generate_responses(model_mistral, prompts_mistral_fewshot)

# 6. Compute metrics (few-shot)
metrics_DeepSeek_fewshot = compute_metrics(responses_DeepSeek_fewshot, y_true)
metrics_mistral_fewshot = compute_metrics(responses_mistral_fewshot, y_true)

print("DeepSeek (few-shot):", metrics_DeepSeek_fewshot)
print("Mistral (few-shot):", metrics_mistral_fewshot)

Generating responses: 100%|██████████| 300/300 [01:58<00:00,  2.54it/s]

DeepSeek (few-shot): {'macro_f1': 0.07878656554712893, 'fail_ratio': 0.10333333333333333}
Mistral (few-shot): {'macro_f1': 0.4748459569093434, 'fail_ratio': 0.0}


## Notes

1. You are free to pick any value for ``num_per_class``.

2. According to our tests, few-shot prompting increases inference time by some minutes (we experimented with ``num_per_class`` $\in [2, 4]$).

# [Task 6 - 1.0 points] Error Analysis

We are now interested in evaluating model responses and comparing their performance.

This analysis helps us in understanding

- Classification task performance gap: are the models good at this task?
- Generation quality: which kind of responses do models generate?
- Errors: which kind of mistakes do models do?

### Instructions

In order to get Task 6 points, we require you to:

* Compare classification performance of selected LLMs in a Table.
* Compute confusion matrices for selected LLMs.
* Briefly summarize your observations on generated responses.

# [Task 7 - 1.0 points] Report

Wrap up your experiment in a short report (up to 2 pages).

### Instructions

* Use the NLP course report template.
* Summarize each task in the report following the provided template.

### Recommendations

The report is not a copy-paste of graphs, tables, and command outputs.

* Summarize classification performance in Table format.
* **Do not** report command outputs or screenshots.
* Report learning curves in Figure format.
* The error analysis section should summarize your findings.

# Submission

* **Submit** your report in PDF format.
* **Submit** your python notebook.
* Make sure your notebook is **well organized**, with no temporary code, commented sections, tests, etc...

# FAQ

Please check this frequently asked questions before contacting us.

### Model cards

You can pick any open-source model card you like.

We recommend starting from those reported in this assignment.

### Implementation

Everything can be done via ``transformers`` APIs.

However, you are free to test frameworks, such as [LangChain](https://www.langchain.com/), [LlamaIndex](https://www.llamaindex.ai/) [LitParrot](https://github.com/awesome-software/lit-parrot), provided that you correctly address task instructions.

### Task Performance

The task is challenging and zero-shot prompting may show relatively low performance depending on the chosen model.

### Prompt Template

Do not change the provided prompt template.

You are only allowed to change it in case of a possible extension.

### Optimizations

Any kind of code optimization (e.g., speedup model inference or reduce computational cost) is more than welcome!

### Bonus Points

0.5 bonus points are arbitrarily assigned based on significant contributions such as:

- Outstanding error analysis
- Masterclass code organization
- Evaluate A1 dataset and perform comparison
- Perform prompt tuning

Note that bonus points are only assigned if all task points are attributed (i.e., 6/6).

# The End